In [661]:
import casadi as ca
import numpy as np

## Numerical solution for ODE 

reference: [Runge-Kutta methods](https://en.wikipedia.org/wiki/Runge%E2%80%93Kutta_methods)

This notebook intends to provide some insights in some numerical integration methods which approximately solve ordinary differential equations.

A differential equation is generally used to describe a dynamic system, such as spring-mass systems and mobile robots. The mathematical formulation of their systems are given either by an explicit ODE or an implicit ODE. Since a close-formed solution of ODE doesn't always exists, we solve the problems leveraging some numerical methods.

**Explicit ODE**: $F\left(x, y, y^{\prime}, \ldots, y^{(n-1)}\right)=y^{(n)}$ and
**Implicit ODE**: $F\left(x, y, y^{\prime}, y^{\prime \prime}, \ldots, y^{(n)}\right)=0$

This part of notebook focuses on the numerical solution of the first order system in the following form: $\dot{\mathbf{x}} = F(t,\mathbf{x})$

### Runge-Kutta order 4

Runge-Kutta order 4(RK4) approximates the next value of $\mathbf{x}(t_{n+1})$ at time step $t_{n+1}$ by calculating the present value $\mathbf{x}_{n}$ plus the weighted average of four increments. 

This method has $O\left(h^5\right)$ local error and $O\left(h^4\right)$ global error.

The time derivatives of $\mathbf{x}$ at the beginning, the midpoint and the end of the interval are determined using:

$\begin{aligned} \mathbf{k}_{1} &=F\left(t_{n}, \mathbf{x}_{n}\right) \\ \mathbf{k}_{2} &=F\left(t_{n}+\frac{h}{2}, \mathbf{x}_{n}+h \frac{\mathbf{k}_{1}}{2}\right) \\ \mathbf{k}_{3} &=F\left(t_{n}+\frac{h}{2}, \mathbf{x}_{n}+h \frac{\mathbf{k}_{2}}{2}\right) \\ \mathbf{k}_{4} &=F\left(t_{n}+h, \mathbf{x}_{n}+h \mathbf{k}_{3}\right) \end{aligned}$, where $h$ is the interval length. The value of next step is then approximated by $\mathbf{x}_{n+1}=\mathbf{x}_{n}+\frac{1}{6} h\left(\mathbf{k}_{1}+2 \mathbf{k}_{2}+2 \mathbf{k}_{3}+\mathbf{k}_{4}\right)$.

In [662]:
def RK4_np(f, x, t, h):
    """
    Runge-Kutta 4th order solver using numpy array data type.

    Args:
        f: A function returning first order ODE in 2D numpy array (Nx x 1).
        x: Current value (list or numpy array). 
        t: Current time.
        h: Step length.
    Returns:
        x_next: Vector of next value in 2D numpy array (Nx x 1)
    """
    x = np.reshape(x, (np.shape(x)[0], -1))    # Reshape x to col vector in np 2D array
    k1 = f(t, x)
    k2 = f(t + h / 2, x + h / 2 * k1)
    k3 = f(t + h / 2, x + h / 2 * k2)
    k4 = f(t + h, x + h * k3)
    x_next = x + h / 6 * (k1 + 2 * k2 + 2 * k3 + k4)
    return x_next

In [629]:
def RK4_ca(f, x, t, h):
    """
    Runge-Kutta 4th order solver using casadi.

    Args:
        f: First order ODE in casadi function (Nx + Nt -> Nx).
        x: Current value (list or numpy array). 
        t: Current time.
        h: Step length.
    Returns:
        x_next: Vector of next value in casadi DM
    """
    k1 = f(t, x)
    k2 = f(t + h / 2, x + h / 2 * k1)
    k3 = f(t + h / 2, x + h / 2 * k2)
    k4 = f(t + h, x + h * k3)
    x_next = x + h / 6 * (k1 + 2 * k2 + 2 * k3 + k4)
    return x_next

Test dynamical system: $\begin{aligned} \dot{x}_1 &= x_2 \\ \dot{x}_2 &= u\end{aligned}$

Analytical solution: $\begin{aligned} x_1 &= x_{10} + x_{20}t + \frac{1}{2}ut^{2} \\ x_2 &= x_{20} + ut\end{aligned}$

In [630]:
def ode_np(t, x):
    '''
    Double integrator
    
    Args:
        t: Current time.
        x: Current value (list or numpy array). 
        
    Returns: First order ODE in 2D numpy array (Nx x 1).
    '''
    x = np.reshape(x, np.shape(x)[0])

    u = 1.0
    x_dot1 = x[1] 
    x_dot2 = u

    return np.array([[x_dot1],[x_dot2]])

In [631]:
x0 = [1.0,2.0]    # Current state
t0 = 1    # Trivial variable time
h = 1  # Step length

#### Numpy RK4 solution

In [632]:
#Solve ODE by RK4
x1 = RK4_np(ode_np, x0, t0, h)
x1

array([[3.5],
       [3. ]])

#### Casadi RK4 solution

In [633]:
def ode_ca(t, x):
    '''
    Double integrator
    
    Args:
        t: Current time.
        x: Current value (list or numpy array). 
        
    Returns: First order ODE in casadi SX col vector .
    '''
    # Parameter konfiguration
    u = 1.0
    rhs = [x[1],
           u
           ]

    return ca.vertcat(*rhs)

In [634]:
Nx = 2    # x dimension: 2
Nt = 1    # t dimension: 1

x = ca.SX.sym('x', Nx)
t = ca.SX.sym('t', Nt)

# Construct a casadi function for the ODE
fn_ca = ca.Function("ode_func", [t, x], [ode_ca(t,x)])

#Solve ODE by RK4
x1 = RK4_ca(fn_ca, x0, t0, h)
x1

DM([3.5, 3])

### Implicit Euler method

The implicit Euler method (also known as backward Euler method) is another numerical methods applied for solving ordinary differential equations. It replaces the slope in the explicit form $F\left(t_{n}, \mathbf{x}_{n}\right)$ by $F\left(t_{n+1}, \mathbf{x}_{n+1}\right)$. The formulation of the implicit Euler method is given by $\mathbf{x}_{n+1}=\mathbf{x}_{n}+h F\left(\mathbf{x}_{n+1}, t_{n+1}\right)$. Since the variable $\mathbf{x}_{n+1}$ exists on both sides of the equation, a numerical root-finding algorithm should be applied to solve the algebraic equation, such as Newton-Raphson method.

- Step 1: Build a Newton-Raphson solver

Newton step update rule: $\delta=-J(x)^{-1} f(x)$

In [658]:
def newton_np(f_newton, J, x, eps = 1e-11):
    '''
    Find a zero of a given function f using the Newton-Raphson method.
 
    Args:
        f_newton: A function of the input x. Function return is in numpy 2D array (Nf x 1).
        J: The Jacobian of f. It is also a function of the input x. Its return is a numpy 2D array (Nf x Nx).
        x: Initial guess of the zero. Better be close to the searched zero to guarantee a better convergence.
        eps: Error tolerance. Stop when |F|_{norm2} < eps.
        
        TODO: stop when the number of iterations is too large
    Returns: The zero of function f in Numpy 2D array (Nx x 1).
    '''
    x_update = np.copy(x)    
    f_value = f_newton(x_update)
    f_tol = 1e6 # dummy initial

    while (f_tol > eps):
        # Update Newton step
        delta = - np.linalg.inv(J(x_update)) @ f_value # Alternative way: using np.linalg.solve(J(x),-f(x))
        x_update += delta
        # Calculate for the cretirion to stop the loop.
        f_diff = f_newton(x_update) - f_value
        f_value = f_newton(x_update)
        f_tol = np.linalg.norm(f_diff) # l2 norm
    return x_update

- Step 2: Construct the ODE solver leveraging implicit Euler method 

The way to calculate the Jacobian is to approximate each element $\frac{\partial f_{i}}{\partial x_{j}}$ in the Jacobian matrix by $\frac{f_{i}(x+\epsilon) - f_{i}(x-\epsilon)}{2\epsilon}$.
The implicit system to be solved is formulated by  $f(\mathbf{x}_{n+1}) = \mathbf{x}_{n+1}-\mathbf{x}_{n}-h F\left(\mathbf{x}_{n+1}, t_{n+1}\right)$.

In [659]:
def Euler_back_np(f, x, t, h, eps = 1e-6):
    """
    Implicit Euler solver using numpy array data type.

    Args:
        f: A function returning first order ODE in 2D numpy array (Nx x 1).
        x: Current value (list or numpy array). 
        t: Current time.
        h: Step length.
        eps: Error tolerence of solving zeros using Newton-Raphson method
    Returns:
        x_next: Vector of next value in 2D numpy array (Nx x 1)
    """
    x = np.reshape(x, (np.shape(x)[0], -1))    # Reshape x to col vector in np 2D array
    
    Nx = np.shape(x)[0]
    def fn_implicit_wrapper(f, x, t_next, h):
        # Return then the function of implicit system to be solved by Newton-Raphson.
        def fn_implicit(x_next):
            f_implicit = x_next - x - h * f(t_next, x_next)
            return f_implicit
        return fn_implicit
    
    def Jacobian_wrapper(fn_implicit, eps_j = 1e-5):
        # Return then the function of implicit system's Jacobian to be used in the Newton-Raphson function.
        def Jacobian(x_next):
            Nx = np.shape(x_next)[0]
            J = np.zeros([Nx,Nx])
            x1 = np.copy(x_next) # TODO: try not allocate to new memory
            x2 = np.copy(x_next) 
            for i in range(Nx):    # go through all x, assign the values vector-wise.
                x1[i][0] += eps_j
                x2[i][0] -= eps_j
                f1 = fn_implicit(x1).flatten()
                f2 = fn_implicit(x2).flatten()
                x1[i][0] -= eps_j
                x2[i][0] += eps_j
                J[: , i] = (f1 - f2)/ (2 * eps_j)
            return J
        return Jacobian
    # Left hand implicit system used to calculate the zero leveraging Newton Raphson method.
    f_lhs = fn_implicit_wrapper(f, x, t+h, h) 
    Jacobian = Jacobian_wrapper(f_lhs)
    x_next = newton_np(f_lhs, Jacobian, x)    # x here is initial guess of x_next, may be assigned to other initial values.
    return x_next

In [660]:
#Solve ODE by implicit Euler method -> obvious lower accuracy compared to RK4 method
x1 = Euler_back_np(ode_np, x0, t0, h)
x1

array([[4.],
       [3.]])

Any convergence problem when h is set too large? -> stiff system.